<a href="https://colab.research.google.com/github/fvianacpm/my-first-repository/blob/master/TFM_1_Preprocessing_FV.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# SUP ML 1 - PREPROCESSING

# Librerias

In [ ]:
import sys
ENV_COLAB = 'google.colab' in sys.modules
if ENV_COLAB:
      !pip install catboost
      !pip install ipywidgets
      !pip install ydata_profiling
      !jupyter nbextension enable --py widgetsnbextension
      print('Environment: Google Colab')

Enabling notebook extension jupyter-js-widgets/extension...
Paths used for configuration of notebook: 
    	/root/.jupyter/nbconfig/notebook.json
Paths used for configuration of notebook: 
    	
      - Validating: OK
Paths used for configuration of notebook: 
    	/root/.jupyter/nbconfig/notebook.json
Environment: Google Colab


In [ ]:
import sys
import time
import warnings

# imports best practice pandas
import os
import numpy as np
import pandas as pd

# imports best practice sklearn
import sklearn
from sklearn import set_config

from sklearn.tree import DecisionTreeClassifier

# transformers
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OrdinalEncoder, OneHotEncoder

# pipelines
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

# model selection e hiperparameters optimization
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV, RepeatedKFold, KFold

from sklearn.metrics import accuracy_score

#Plot and profiling
import matplotlib.pyplot as plt
import seaborn as sns
from ydata_profiling import ProfileReport
from catboost import CatBoostClassifier, Pool
from ydata_profiling import ProfileReport

pd.set_option('display.max_columns',100)
pd.set_option('display.max_rows', None)

import warnings
warnings.filterwarnings("ignore")

# Importar Data

agregar por mes/año
producto/tienda mes/año
analisis sobre las ventas
olvidar de eventos ---no es relevante
Productos mas vendidos y los menos vendidos
Nada de cantidad
groupby para la cantiudad un sum y para el precio un mean

Portada
Indice
Slide 1 overview por Grafico barras por tienda
Slide 2 con mas detlles
Slide 3 productos mas y menos veindo
evitar verde sobre blanco
__________
Clustering
Clustering, cada producto es una fila (3049 filas), no coger cada producto tienda
precio medio
cuantas ventas ha tenido en cuanto a uds y dinero.
Precio/cantidas
Variacion que ha tenido el precio desde el principio hasta el final
4 clusters
Entender que tiene en comun cada cluster. Poner una explicacion como hemos hecho los clusters, que tienen en comun, que los diferencia. Nombre a cada cluster(y sus caracteristicas pr) y que cada observacion sea de un producto diferente

In [ ]:
ENV_COLAB = 'google.colab' in sys.modules
if ENV_COLAB:
  from google.colab import drive
  drive.mount('/content/drive')
else:
  print('Environment: Local')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
ENV_COLAB = 'google.colab' in sys.modules
if ENV_COLAB:
  df_p = pd.read_csv(r'/content/drive/MyDrive/TFM/CSV/item_prices.csv')
  df_s=pd.read_csv(r'/content/drive/MyDrive/TFM/CSV/item_sales.csv')
  df_c=pd.read_csv(r'/content/drive/MyDrive/TFM/CSV/daily_calendar_with_events.csv')
else:
  df_p = pd.read_csv(r'CSV/item_prices.csv')
  df_s=pd.read_csv(r'CSV/item_sales.csv')
  df_c=pd.read_csv(r'CSV/daily_calendar_with_events.csv')

EDA Price

In [ ]:
# Display df_p in a table
print("Price")
df_p.sample(5)



Price


,item,category,store_code,yearweek,sell_price
1112634,SUPERMARKET_1_115,SUPERMARKET,NYC_2,201448.0,4.2960
1914994,SUPERMARKET_3_169,SUPERMARKET,NYC_3,201341.0,4.7760
6610895,HOME_&_GARDEN_2_434,HOME_&_GARDEN,PHI_3,201203.0,2.3500
781908,ACCESORIES_1_310,ACCESORIES,NYC_2,201424.0,6.6234
6961091,SUPERMARKET_3_808,SUPERMARKET,PHI_3,201313.0,1.2000


In [ ]:
profile_price = ProfileReport(df_p, title="Profiling Report - Price", minimal=True)
profile_price.to_notebook_iframe()

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
df_p1=df_p.copy()

In [ ]:
df_p1.dropna(subset=['sell_price'], inplace=True)

In [ ]:
#Drop null rows cause as menthioned,price for the product “item” for the period in “yearweek”.
#Prices are provided per week (average across 7 days). If not available, there were no sales
#for the product during that week. If there was no sales in the week, makes no sanse to have a price register.
df_p1['yearweek']= df_p1['yearweek'].fillna(method='bfill')
df_p1['yearweek']= df_p1['yearweek'].fillna(method='ffill').astype(int)

In [ ]:
# Convert 'year_week' to a string and then to datetime

df_p1['yearweek'] = df_p1['yearweek'].astype(str) + '0' # Add '0' to represent the first day of the week
df_p1['date'] = pd.to_datetime(df_p1['yearweek'].str[:4] + df_p1['yearweek'].str[4:], format='%Y%U%w')

# Extract year and month
df_p1['year'] = df_p1['date'].dt.year
df_p1['month'] = df_p1['date'].dt.month

 # Create_Year_Month
df_p1['yearmonth'] = df_p1['year'].astype(str) + df_p1['month'].astype(str).str.zfill(2)

# Drop the temporary 'date' column if not needed
df_p1.drop(columns=['date'], inplace=True)
df_p1.drop(columns=['year'], inplace=True)
df_p1.drop(columns=['month'], inplace=True)
df_p1.drop(columns=['yearweek'], inplace=True)

#add pk
df_p1['id_p']= df_p1['yearmonth'].astype(str) + '_' + df_p1['item'].astype(str) + '_' + df_p1['store_code'].astype(str)

In [ ]:
df_p1.head(10)

,item,category,store_code,sell_price,yearmonth,id_p
0,ACCESORIES_1_001,ACCESORIES,NYC_1,12.7414,201307,201307_ACCESORIES_1_001_NYC_1
1,ACCESORIES_1_001,ACCESORIES,NYC_1,12.7414,201307,201307_ACCESORIES_1_001_NYC_1
2,ACCESORIES_1_001,ACCESORIES,NYC_1,10.9858,201307,201307_ACCESORIES_1_001_NYC_1
3,ACCESORIES_1_001,ACCESORIES,NYC_1,10.9858,201308,201308_ACCESORIES_1_001_NYC_1
4,ACCESORIES_1_001,ACCESORIES,NYC_1,10.9858,201308,201308_ACCESORIES_1_001_NYC_1
5,ACCESORIES_1_001,ACCESORIES,NYC_1,10.9858,201308,201308_ACCESORIES_1_001_NYC_1
6,ACCESORIES_1_001,ACCESORIES,NYC_1,10.9858,201308,201308_ACCESORIES_1_001_NYC_1
7,ACCESORIES_1_001,ACCESORIES,NYC_1,10.9858,201309,201309_ACCESORIES_1_001_NYC_1
8,ACCESORIES_1_001,ACCESORIES,NYC_1,10.9858,201309,201309_ACCESORIES_1_001_NYC_1
9,ACCESORIES_1_001,ACCESORIES,NYC_1,10.9858,201309,201309_ACCESORIES_1_001_NYC_1


In [ ]:
gdf_p1 = df_p1.groupby(['yearmonth', 'category','store_code','item','id_p']).agg(
    AVG_sell_price=('sell_price', np.mean)
).reset_index()
gdf_p1.head(10)

,yearmonth,category,store_code,item,id_p,AVG_sell_price
0,201101,ACCESORIES,BOS_1,ACCESORIES_1_003,201101_ACCESORIES_1_003_BOS_1,3.950100
1,201101,ACCESORIES,BOS_1,ACCESORIES_1_004,201101_ACCESORIES_1_004_BOS_1,5.772200
2,201101,ACCESORIES,BOS_1,ACCESORIES_1_007,201101_ACCESORIES_1_007_BOS_1,10.480400
3,201101,ACCESORIES,BOS_1,ACCESORIES_1_008,201101_ACCESORIES_1_008_BOS_1,0.635444
4,201101,ACCESORIES,BOS_1,ACCESORIES_1_009,201101_ACCESORIES_1_009_BOS_1,2.157556
5,201101,ACCESORIES,BOS_1,ACCESORIES_1_010,201101_ACCESORIES_1_010_BOS_1,4.216100
6,201101,ACCESORIES,BOS_1,ACCESORIES_1_011,201101_ACCESORIES_1_011_BOS_1,4.628400
7,201101,ACCESORIES,BOS_1,ACCESORIES_1_012,201101_ACCESORIES_1_012_BOS_1,7.953400
8,201101,ACCESORIES,BOS_1,ACCESORIES_1_014,201101_ACCESORIES_1_014_BOS_1,1.542800
9,201101,ACCESORIES,BOS_1,ACCESORIES_1_015,201101_ACCESORIES_1_015_BOS_1,0.931000


In [ ]:
print(gdf_p1.duplicated().sum())


0


In [ ]:
#How many rows did we grouped?
print(gdf_p1.shape)
print(df_p1.shape)


(1550184, 6)
(6965706, 6)


In [ ]:
#Check if we only have the data from Phili/NY/BO
print(gdf_p1['store_code'].unique())
print(df_p1['store_code'].unique())


['BOS_1' 'BOS_2' 'BOS_3' 'NYC_1' 'NYC_2' 'NYC_3' 'NYC_4' 'PHI_1' 'PHI_2'
 'PHI_3']
['NYC_1' 'NYC_2' 'NYC_3' 'NYC_4' 'BOS_1' 'BOS_2' 'BOS_3' 'PHI_1' 'PHI_2'
 'PHI_3']


EDA Calendar


In [ ]:
profile_Calendar = ProfileReport(df_c, title="Profiling Report - Calendar", minimal=True)
profile_Calendar.to_notebook_iframe()

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
df_c.drop(columns=['event'], inplace=True)

In [ ]:
# Display df_c in a table
print("Calendar")
df_c.sample(5)

Calendar


,date,weekday,weekday_int,d
1233,2014-06-15,Sunday,2,d_1234
1769,2015-12-03,Thursday,6,d_1770
367,2012-01-31,Tuesday,4,d_368
1072,2014-01-05,Sunday,2,d_1073
1199,2014-05-12,Monday,3,d_1200


In [ ]:
df_c1=df_c.copy()

In [ ]:
df_c1['date'] = pd.to_datetime(df_c1['date'])


In [ ]:
# Extract year and month
df_c1['year'] = df_c1['date'].dt.year
df_c1['month'] = df_c1['date'].dt.month

 # Create_Year_Month
df_c1['yearmonth'] = df_c1['year'].astype(str) + df_c1['month'].astype(str).str.zfill(2)

# Drop the temporary 'date' column if not needed
df_c1.drop(columns=['date'], inplace=True)
df_c1.drop(columns=['year'], inplace=True)
df_c1.drop(columns=['month'], inplace=True)
df_c1.drop(columns=['weekday_int'], inplace=True)
df_c1.drop(columns=['weekday'], inplace=True)

print(df_c1.head(10))

      d yearmonth
0   d_1    201101
1   d_2    201101
2   d_3    201101
3   d_4    201102
4   d_5    201102
5   d_6    201102
6   d_7    201102
7   d_8    201102
8   d_9    201102
9  d_10    201102


EDA DF Sales

In [ ]:
df_s.shape

(30490, 1920)

In [ ]:
df_s1=df_s.copy()

In [ ]:
cols = [col for col in df_s.columns if col.startswith('d_')]
df_s1 = df_s1.melt(id_vars=[col for col in df_s.columns if col not in cols],
                      value_vars=cols,
                      var_name='d_column',
                      value_name='d_value')

In [ ]:
df_s1.sample(5)

In [ ]:
df_s1 = df_s1[df_s1['d_value'] != 0]

In [ ]:
df_s1.shape


In [ ]:
profile_sales = ProfileReport(df_s1, title="Profiling Report - Sales", minimal=True)
profile_sales.to_notebook_iframe()

In [ ]:
print('Calendar Shape:',df_c1.shape)
print('\nSales:',df_s1.shape)
print('\nPrice:',gdf_p1.shape)

In [ ]:
df_s_c = df_s1.merge(df_c1, left_on='d_column', right_on='d', how='left')

In [ ]:
df_s_c['id_p']= df_s_c['yearmonth'].astype(str) + '_' + df_s_c['id'].astype(str)

In [ ]:
df_s_c.head(10)

In [ ]:
df_s_c1=df_s_c.copy()

In [ ]:
df_s_c1.drop(columns=['d_column'], inplace=True)
df_s_c1.drop(columns=['d_value'], inplace=True)
df_s_c1.drop(columns=['d'], inplace=True)

In [ ]:
gdf_p1.head(1)

,yearmonth,category,store_code,item,id_p,AVG_sell_price
0,201101,ACCESORIES,BOS_1,ACCESORIES_1_003,201101_ACCESORIES_1_003_BOS_1,3.9501


In [ ]:
df_s_c.isnull().sum()

id            0
item          0
category      0
department    0
store         0
store_code    0
region        0
d_column      0
d_value       0
d             0
yearmonth     0
id_p          0
dtype: int64

In [ ]:
gdf_s_c = df_s_c.groupby(['yearmonth', 'category','store_code','item','id_p']).agg(
    AVG_sell_price=('sell_price', np.mean)
).reset_index()
gdf_p1.head(10)

In [ ]:
df_p1.columns

Index(['item', 'category', 'store_code', 'yearweek', 'sell_price', 'id',
       'id_p'],
      dtype='object')

In [ ]:
df_p2=df_p1[['yearweek','sell_price','id_p']]

In [ ]:
df_p2.sample(5)

,yearweek,sell_price,id_p
623538,201445,2.976,SUPERMARKET_3_445_NYC_1_201445
2496246,201345,8.688,SUPERMARKET_2_068_NYC_4_201345
5560776,201512,2.016,SUPERMARKET_3_802_PHI_1_201512
1837908,201545,3.072,SUPERMARKET_2_232_NYC_3_201545
3676975,201442,4.925,HOME_&_GARDEN_1_338_BOS_2_201442


In [ ]:
df_s_c.to_csv('df_s_c.csv', index=False)

In [ ]:
df_p2.to_csv('df_p2', index=False)

# EDA: Analisis Exploratorio

## Distribución del target

In [ ]:
def find_primary_key(df):
    for col in df.columns:
        if df[col].duplicated().sum() == 0:
            return col
    return None

In [ ]:
print('Price primary key:', find_primary_key(df_p1))
print('Sales primary key:', find_primary_key(df_s1))
print('Calendar primary key:', find_primary_key(df_c))

## Analisis Univariable

* Distribucion de cada variable

## Analisis Variable-Target

* Relacion target-variable

# Data Cleaning

* Eliminar filas sin target informado.
* Eliminar filas duplicadas (si tenemos ID, solo puede haber 1 registro por ID)
* Eliminar filas/columnas vacías y columnas irrelevantes
* Corregir data types incorrectos
* Categoricos: Corregir literales incorrectos (acentos, erratas, etc)
* Fechas y textos: convertir a variables numericas (hour, day, month, year...) o categoricas (weekday_name...)

# Imputación de nulos

* MODELOS DE ARBOLES: Imputar valor outlier.
* MODELOS LINEALES: imputar media, mediana o moda.
* Si vamos a practicar un Torneo de Modelos, preferible imputar valor outlier.

# Encoding categóricos

* Si tienen sentido ordinal, es decir, se pueden ordenar, encodear con una escala numerica. Ejemplo: {"Bajo":1, "Medio":2, "Alto":3}.
* Si tienen unicamente 2 clases, indicador booleano
* One Hot Encoding en resto de categoricos

# Eliminar alta correlación

* Analizar variables (X) altamente correlacionadas
* Eliminar una variable de cada pareja altamente correlacionada (>95% o >99%...) en el dataset (df)

# Eliminar baja varianza

* Eliminar variables (X) practicamente constantes con un threshold minimo (1% o menos)

# Guardar clean data

* Guardar en data path y mostrar el resultado en un head()